# Load Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import joblib
import tensorflow as tf
from tensorflow import keras 
from keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import initializers
from tensorflow.keras import regularizers

from keras.layers.core import Dense, Dropout, Activation
import keras.callbacks as callbacks
from keras.utils import np_utils

# Load Data

In [37]:
df_config = pd.read_csv('../data/config.csv')
df_config

,Train Label,Raw Data Path,Info Files Path,MFCC Processed Path,Classes Name Path,ID 2 Class Path,Ready to Train Data Path,Indexes Path,Number of Folds,Pipeline Path,Model Path,Hidden Neurons Path,Number of Inits
0,ShipsEar_NN_Classification,../data/shipsEar_AUDIOS,../data/file_info.csv,../data/mfcc_data.csv,../data/class_names.csv,../data/id2classes.csv,../data/train_data.csv,../data/indexes,5,../data/pipelines,../data/models,../data/models/hidden_neurons.pkl,1


In [3]:
df_train = pd.read_csv(df_config['Ready to Train Data Path'][0])

## Fit a preprocessing pipeline for each kFold

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

index_path = df_config['Indexes Path'][0]
pipe_path = df_config['Pipeline Path'][0]
n_folds =  df_config['Number of Folds'][0]

data = df_train.drop(columns=['target'])

for ifold in range(n_folds):
    index_file = 'kFoldsCV_fold_%i_of_%i_indexes.pkl'%(ifold, n_folds)
    with open(os.path.join(index_path,index_file),'rb') as file_handler:
            [trn_idx,val_idx] = pickle.load(file_handler)
    
    # criando o pipeline
    pipe = Pipeline(steps=[("scaler", MinMaxScaler())])
    pipe.fit(data.loc[trn_idx,:])
    
    pipe_name = 'kFoldsCV_fold_%i_of_%i_pipe.pkl'%(ifold, n_folds)
    with open(os.path.join(pipe_path,pipe_name),'wb') as file_handler:
        joblib.dump(pipe, file_handler)

# Define Model Class

In [4]:
class MLPModel:
    def __init__(self, n_hidden_neurons=2, verbose=2):
        self.n_hidden_neurons = n_hidden_neurons
        self.model = None
        self.trn_history = None
        self.trained = False
        self.verbose = verbose
    def __str__(self):
        m_str = 'Class MLPModel\n'
        if self.trained:
            m_str += 'Model is fitted, '
        else:
            m_str += 'Model is not fitted, '
        m_str += 'instance created with %i hidden neurons'%(self.n_hidden_neurons) 
        return m_str
    def create_model(self, data, target, random_state=0, learning_rate=0.01):
        #tf.random.set_seed(random_state)

        model = tf.keras.Sequential()
        
        # add a input to isolate the input of NN model
        model.add(tf.keras.Input(shape=(data.shape[1],)))
        # add a non-linear single neuron layer
        hidden_layer = layers.Dense(units=self.n_hidden_neurons,
                                    activation='tanh',
                                    kernel_initializer=initializers.RandomNormal(stddev=0.01),
                                    kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
                                    bias_regularizer=regularizers.L2(1e-4),
                                    bias_initializer=initializers.Zeros()
                                   )
        model.add(hidden_layer)
        # add a non-linear output layer with max sparse target shape
        output_layer = layers.Dense(units=target.shape[1],
                                    activation='tanh',
                                    kernel_initializer=initializers.RandomNormal(stddev=0.01),
                                    bias_initializer=initializers.Zeros()
                                   )
        model.add(output_layer)
        # creating a optimization function using steepest gradient
        lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=learning_rate,
                                                                  decay_steps=100,
                                                                  decay_rate=0.9)
        optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)

        # compile the model
        loss = keras.losses.CategoricalCrossentropy(from_logits=False,
                                                    label_smoothing=0.0,
                                                    axis=-1,
                                                    reduction="auto",
                                                    name="cat_crossent",)


        cat_cross = keras.losses.BinaryCrossentropy(from_logits=False,
                                                         label_smoothing=0.0,
                                                         axis=-1,
                                                         reduction="auto",
                                                         name="cat_crossent_met",)
        cat_acc_metric = keras.metrics.BinaryAccuracy(name="cat_acc", dtype=None)
        acc_metric = keras.metrics.Accuracy(name="accuracy",dtype=None)
        mse_metric = keras.metrics.MeanSquaredError(name="mse", dtype=None)
        rmse_metric = keras.metrics.RootMeanSquaredError(name="rmse", dtype=None)

        model.compile(loss="mean_squared_error", 
                      optimizer=optimizer,
                      metrics=[acc_metric,mse_metric,rmse_metric])
        return model
    def fit(self, X, Y,
            trn_id=None, 
            val_id=None, 
            epochs=50,
            batch_size=4,
            patience = 100,
            learning_rate=0.01, random_state=0):
        
        X_copy = X.copy()
        Y_copy = Y.copy()
        
        model = self.create_model(X_copy,Y_copy, random_state=random_state, learning_rate=learning_rate)
        
        # early stopping to avoid overtraining
        earlyStopping = callbacks.EarlyStopping(monitor='val_loss', 
                                                patience=patience,verbose=self.verbose, 
                                                mode='auto')
    
        trn_desc = model.fit(X_copy[trn_id,:], Y_copy[trn_id],
                             epochs=epochs,
                             batch_size=batch_size,
                             callbacks=[earlyStopping], 
                             verbose=self.verbose,
                             validation_data=(X_copy[val_id,:],
                                              Y_copy[val_id]),
                            )
        self.model = model
        self.trn_history = trn_desc
        self.trained = True
    def predict(self, data):
        return self.model.predict(data)
    def save(self, file_path):
        with open(file_path,'wb') as file_handler:
            joblib.dump([self.n_hidden_neurons, self.model,
                        self.trn_history, self.trained], file_handler)
    def load(self, file_path):
        with open(file_path,'rb') as file_handler:
            [self.n_hidden_neurons, self.model, self.trn_history, self.trained]= joblib.load(file_handler)
    def model_with_no_output_layer(self):
        buffer_model = tf.keras.Sequential()    
        # add a input to isolate the input of NN model
        buffer_model.add(tf.keras.Input(shape=(model.model.layers[0].get_weights()[0].shape[0],)))
        # add a non-linear single neuron layer
        hidden_layer = layers.Dense(units=model.model.layers[0].get_weights()[1].shape[0],
                                    activation='tanh')
        buffer_model.add(hidden_layer)    
        output_layer = layers.Dense(units=1,activation='tanh')
    
        for idx, layer in enumerate(buffer_model.layers):
            layer.set_weights(model.model.layers[idx].get_weights())
        return buffer_model
    def predict_one_layer_before_output(self, data):
        buffer_model = self.model_with_no_output_layer()
        return buffer_model.predict(data)


In [40]:
# descomente somente quando for necessário
#hidden_neurons = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
#hidden_neurons = [1]
#write_list_of_hidden_neurons(df_config['Hidden Neurons Path'][0],hidden_neurons)

# Config for kFold training

In [41]:
hidden_neurons = get_list_of_hidden_neurons(df_config['Hidden Neurons Path'][0])
model_path = df_config['Model Path'][0]
n_folds =  df_config['Number of Folds'][0]

print('kFold Files Generation')

buffer = 0*np.ones([n_folds,len(hidden_neurons)])
df_buffer = pd.DataFrame(data=buffer, columns=hidden_neurons,index=range(n_folds))
df_buffer.to_csv(os.path.join(model_path, '%s_kfold_model_acc.csv'%(df_config['Train Label'][0])))
buffer = False*np.ones([n_folds,len(hidden_neurons)])
df_buffer = pd.DataFrame(data=buffer, columns=hidden_neurons,index=range(n_folds))
df_buffer.to_csv(os.path.join(model_path, '%s_kfold_model_status.csv'%(df_config['Train Label'][0])))
buffer = False*np.ones([n_folds,len(hidden_neurons)])
df_buffer = pd.DataFrame(data=buffer, columns=hidden_neurons,index=range(n_folds))
df_buffer.to_csv(os.path.join(model_path, '%s_kfold_model_names.csv'%(df_config['Train Label'][0])))

kFold Files Generation


In [45]:
# for kFolds CV

from sklearn.metrics import accuracy_score

print('kFold Training Process')

if True: # remove when your are shure to spent time!!!
    index_path = df_config['Indexes Path'][0]
    pipe_path = df_config['Pipeline Path'][0]
    model_path = df_config['Model Path'][0]

    data = df_train.drop(columns=['target']).values
    trgt = df_train['target'].values
    
    df_acc = pd.read_csv(os.path.join(model_path, '%s_kfold_model_acc.csv'%(df_config['Train Label'][0])),index_col=0)
    df_status = pd.read_csv(os.path.join(model_path, '%s_kfold_model_status.csv'%(df_config['Train Label'][0])),index_col=0)
    df_status = df_status.astype(bool)
    df_names = pd.read_csv(os.path.join(model_path, '%s_kfold_model_names.csv'%(df_config['Train Label'][0])),index_col=0)
    df_names = df_names.astype(str)

    for ifold in range(n_folds):
        print('Training Process %i fold of %i folds'%(ifold, n_folds))
        cv_name = 'kFoldsCV_fold_%i_of_%i_indexes.pkl'%(ifold, n_folds)
        with open(os.path.join(index_path,cv_name),'rb') as file_handler:
            [trn_idx,val_idx] = pickle.load(file_handler)
        pipe_name = 'kFoldsCV_fold_%i_of_%i_pipe.pkl'%(ifold, n_folds)
        with open(os.path.join(pipe_path,pipe_name),'rb') as file_handler:
            pipe = joblib.load(file_handler)
        
        trn_data = pipe.transform(data)
        trn_trgt = tf.keras.utils.to_categorical(trgt, num_classes=len(np.unique(trgt)))
        
        for ineuron in hidden_neurons:
            print('Training for %i neuron in'%(ineuron),hidden_neurons)
            best_acc = -1
            best_init = -1
            if df_status.loc[ifold,str(ineuron)] == False:
                 for iinit in range(df_config['Number of Inits'][0]):
                        print('%s - Training Model with %i neurons - %i init'%(df_config['Train Label'][0], ineuron, iinit))
                        model_name = '%s_kFoldsCV_fold_%i_MLPModel_%i_hidden_neurons_%i_init.pkl'%(df_config['Train Label'][0], 
                                                                                                   ifold, ineuron,
                                                                                                   iinit)
                        if os.path.exists(os.path.join(model_path, model_name)) == False:
                            model = MLPModel(n_hidden_neurons=ineuron,verbose=2)
                            model.fit(trn_data, trn_trgt, trn_id=trn_idx, val_id=val_idx, epochs=5, random_state=iinit)
                            model.save(os.path.join(model_path, model_name))
                        else:
                            model = MLPModel(n_hidden_neurons=ineuron,verbose=0)
                            model.load(os.path.join(model_path, model_name))
                        model_acc = accuracy_score(trn_trgt,np.sign(model.predict(pipe.transform(data))))

kFold Training Process
Training Process 0 fold of 5 folds
Training for 1 neuron in [1]
ShipsEar_NN_Classification - Training Model with 1 neurons - 0 init


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Epoch 1/1000
115934/115934 - 148s - loss: 0.0640 - accuracy: 0.0000e+00 - mse: 0.0638 - rmse: 0.2527 - val_loss: 0.0683 - val_accuracy: 0.0000e+00 - val_mse: 0.0681 - val_rmse: 0.2610 - 148s/epoch - 1ms/step
Epoch 2/1000
115934/115934 - 159s - loss: 0.0640 - accuracy: 0.0000e+00 - mse: 0.0638 - rmse: 0.2526 - val_loss: 0.0683 - val_accuracy: 0.0000e+00 - val_mse: 0.0681 - val_rmse: 0.2610 - 159s/epoch - 1ms/step
Epoch 3/1000


KeyboardInterrupt: 

In [31]:
trn_trgt

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)